In [2]:
from bs4 import BeautifulSoup
import requests

In [3]:
#retina - emprendimiento
url = 'https://retina.elpais.com/tag/iniciativa_empresarial/a/'


In [15]:
index = ['0', '1']
title = []
link = []
date = []
section = []
autor = []

#beafigueroa: Buscamos el botón >
res_pag = requests.get(url)
soup_pag = BeautifulSoup(res_pag.text, 'lxml')

#El botón está en <li class="paginacion-siguiente activo"><a href="XXXXXXX">Siguiente ›</a></li>
link_pag = soup_pag.find('li', {'class': 'paginacion-siguiente activo'}).a['href'] 

#Al final de la cadena del link está el número de páginas que tiene la sección
num_pag = int(link_pag.split('/')[-1])
index = range(num_pag+1) #El orden real de páginas tendría que ser [0,5,4,3,2,1] pero así funciona.
print num_pag, index

#Para PRUEBAS, solo una página.
index = ['0']

for number in index:
    res = requests.get(url+str(number)) #beafigueroa: Index ahora tiene int, para concatenar lo cambio a str
    soup = BeautifulSoup(res.text, 'lxml')
    
    title_test = soup.find_all('h2', {'class': 'articulo-titulo'})


    for word in title_test:
        new_title = word.text
        title.append(new_title)
        #beafigueroa: Recoge el link al artículo y dentro del link recoge la fecha    
        new_link = "https://retina.elpais.com"+word.a['href'] 
        res_link = requests.get(new_link)
        soup_link = BeautifulSoup(res_link.text, 'lxml')
        new_date = soup_link.time['datetime']
        link.append(new_link)
        date.append(new_date)
    
    art = soup.find_all('div', {'class':'articulo-antetitulo'})

    for news in art:
        new_section = news.text
        section.append(new_section.replace("\n", "")) #le quitamos \n para que no salte de línea el csv
        
    autor_test = soup.find_all('div', {'class':'autor-texto'})

    for name in autor_test:
        new_name = name.text
        autor.append(new_name.replace("\n", "")) #le quitamos \n para que no salte de línea el csv
  

print len(title)
print len(link)
print len(date)
print len(section)
print len(autor)


5 [0, 1, 2, 3, 4, 5]
33
33
33
33
34


In [5]:
# ¡¡FUNCIONA PARA PASAR A XLS!! Falta autor

import pandas as pd
data = {'Title':title, 
        'PageLink':link,
        'Date':date,
        'Section':section
        }
#        'Author':autor
news = pd.DataFrame(data=data)
news.head(5)
#cols = ['Title', 'PageLink', 'Date', 'Section', 'Author']
cols = ['Title', 'PageLink', 'Date', 'Section']
news = news[cols]
#retina_news = oldnews.append(news)
retina_news = news
filename = 'retina_news1.xlsx'
wks_name = 'Data'
writer = pd.ExcelWriter(filename)
retina_news.to_excel(writer, wks_name, index=False)
writer.save()

In [16]:
# ¡¡FUNCIONA PARA PASAR A CSV!! Falta autor

import pandas as pd
import csv
data = {'Title':title, 
        'PageLink':link,
        'Date':date,
        'Section':section
        }
#        'Author':autor

news = pd.DataFrame(data=data)

#cols = ['Title', 'PageLink', 'Date', 'Section', 'Author']
cols = ['Title', 'PageLink', 'Date', 'Section']
news = news[cols]

filename = 'retina_news.csv'

news.to_csv(filename, index=False, encoding='utf-8-sig', header=True, sep='|')
